# 02 – Feature Engineering for US YouTube Trending Videos

This notebook performs feature engineering for the US YouTube trending videos
dataset and prepares two processed datasets:

1. `../data/processed/features.csv` – base features for each video-day.
2. `../data/processed/features_with_trends.csv` – the same feature set enriched
   with category-level Google Trends scores.

The output of this notebook is used later in `03_modeling.ipynb`.


## 1. Imports

In [ ]:
# 02_feature_engineering.ipynb
# Core feature engineering steps for the US YouTube Trending dataset

import pandas as pd
import numpy as np

print("Feature engineering notebook ready.")

## 2. Load Raw Data and Basic Ratios

We start from the raw US YouTube trending dataset stored at
`../data/raw/USvideos.csv`. For convenience, we also recompute the basic
engagement ratios so that this notebook can be run standalone.


In [ ]:
# Load raw US YouTube trending data
df = pd.read_csv("../data/raw/USvideos.csv")

# Recreate ratios in case this notebook is run standalone
df["like_view_ratio"] = df["likes"] / (df["views"] + 1e-6)
df["comment_view_ratio"] = df["comment_count"] / (df["views"] + 1e-6)

df.head()

## 3. Publish Time Parsing

The `publish_time` column is parsed into:

- `publish_time` (datetime)
- `publish_date` (date only)
- `publish_hour` (hour of day)

These are useful temporal features for modeling.


In [ ]:
# Parse publish_time into datetime, date and hour components
df["publish_time"] = pd.to_datetime(df["publish_time"], errors="coerce")
df["publish_date"] = df["publish_time"].dt.date
df["publish_hour"] = df["publish_time"].dt.hour

df[["publish_time", "publish_date", "publish_hour"]].head()

## 4. Fix `trending_date` Format

In the original Kaggle dataset, the `trending_date` column is stored as
`YY.DD.MM`, for example `'17.14.11'` meaning **2017‑11‑14**.

We convert it into a proper `datetime` column.


In [ ]:
# Fix trending_date format: original is 'YY.DD.MM', e.g. '17.14.11' meaning 2017-11-14
def fix_trending_date(x: str) -> str:
    yy, dd, mm = x.split(".")
    return f"20{yy}-{mm}-{dd}"

df["trending_date_fixed"] = df["trending_date"].apply(fix_trending_date)
df["trending_date"] = pd.to_datetime(df["trending_date_fixed"], errors="coerce")
df = df.drop(columns=["trending_date_fixed"])

df[["trending_date"]].head()

## 5. Next-Day Views and Growth Metrics

To understand how fast a video is growing, we compare the current day's view
count with the **next trending day's** view count for the same video.

Steps:

1. Sort by `video_id` and `trending_date`.
2. For each `video_id`, shift the `views` column by -1 to get `views_next_day`.
3. Compute absolute and relative growth:
   - `view_growth = views_next_day - views`
   - `growth_rate = view_growth / views`


In [ ]:
# Sort by video_id and trending_date so that we can compute next-day views
df = df.sort_values(by=["video_id", "trending_date"])
df[["video_id", "trending_date", "views"]].head(10)

In [ ]:
# Compute next-day view count per video
df["views_next_day"] = df.groupby("video_id")["views"].shift(-1)

df[["video_id", "trending_date", "views", "views_next_day"]].head(10)

In [ ]:
# Compute absolute and relative growth
df["view_growth"] = df["views_next_day"] - df["views"]
df["growth_rate"] = df["view_growth"] / (df["views"] + 1e-6)

df[["views", "views_next_day", "view_growth", "growth_rate"]].head(10)

## 6. Define the High-Growth Label

We restrict ourselves to rows where `growth_rate` is defined (i.e. not the last
trending day of each video), and define a **binary label**:

- `high_growth = 1` if `growth_rate` is in the top 25% (≥ 75th percentile).
- `high_growth = 0` otherwise.

This label is what we predict later in `03_modeling.ipynb`.


In [ ]:
# Drop rows where growth_rate is NaN (typically the last trending day of each video)
df_valid = df.dropna(subset=["growth_rate"]).copy()

# Define high_growth label as top 25% of growth_rate
threshold = df_valid["growth_rate"].quantile(0.75)
df_valid["high_growth"] = (df_valid["growth_rate"] >= threshold).astype(int)

print("High growth threshold (75th percentile):", threshold)
df_valid["high_growth"].value_counts()

## 7. Select Feature Columns and Save `features.csv`

We keep the following columns as our *base* feature set for modeling and save
them to `../data/processed/features.csv`.


In [ ]:
# Select feature columns for modeling
feature_cols = [
    "video_id",
    "trending_date",
    "publish_date",
    "publish_hour",
    "views",
    "likes",
    "dislikes",
    "comment_count",
    "like_view_ratio",
    "comment_view_ratio",
    "view_growth",
    "growth_rate",
    "high_growth",
    "category_id",
]

features = df_valid[feature_cols].copy()
print("Processed features shape:", features.shape)
features.head()

In [ ]:
# Save processed features
features.to_csv("../data/processed/features.csv", index=False)
print("Saved processed features to ../data/processed/features.csv")

## 8. Enrich Features with Google Trends

Finally, we merge the base `features.csv` with the category-level Google Trends
data stored in `../data/raw/google_trends_category.csv`.

The merge is done on:

- `trending_date` (from the features table)
- `category_id` (YouTube category)

We also compute 3-day and 7-day rolling averages of the trend score per
category to smooth out noise.


In [ ]:
# Load base features table (one row per video-day)
df_feat = pd.read_csv(
    "../data/processed/features.csv",
    parse_dates=["trending_date", "publish_date"],
)

print("Base features shape:", df_feat.shape)

# Load Google Trends data
trends = pd.read_csv(
    "../data/raw/google_trends_category.csv",
    parse_dates=["date"],
)

print("Google Trends shape:", trends.shape)

# Join on date and category_id
merged = df_feat.merge(
    trends,
    left_on=["trending_date", "category_id"],
    right_on=["date", "category_id"],
    how="left",
)

# Drop helper columns not needed anymore
merged = merged.drop(columns=["date", "keyword"], errors="ignore")

print("Merged shape (features + trends):", merged.shape)
merged.head()

In [ ]:
# Compute rolling mean trend scores per category
merged = merged.sort_values(["category_id", "trending_date"])

merged["trend_score_3d_mean"] = (
    merged.groupby("category_id")["trend_score"]
    .transform(lambda s: s.rolling(window=3, min_periods=1).mean())
)

merged["trend_score_7d_mean"] = (
    merged.groupby("category_id")["trend_score"]
    .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
)

# Save final feature set enriched with trends
merged.to_csv("../data/processed/features_with_trends.csv", index=False)
print("Saved features_with_trends to ../data/processed/features_with_trends.csv")

merged.head()